<a href="https://colab.research.google.com/github/irohan98/Agents-Practice/blob/main/recipe_ingestion_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Packages

In [ ]:
# Install packages
!pip install openai-whisper yt-dlp sentence-transformers pandas -q
!apt-get update && apt-get install -y ffmpeg -q  # (For Linux/Colab systems)
!pip install youtube-transcript-api --upgrade

In [ ]:
import os
import re
import json
import requests
from urllib.parse import urlparse
from typing import Dict, List
from IPython.display import display, Markdown

import pandas as pd
import whisper
import yt_dlp
from sentence_transformers import SentenceTransformer

### Load Model

In [ ]:
# Load Whisper model for audio transcription
whisper_model = whisper.load_model("base")

# Load SentenceTransformer for embedding recipes
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Recipe store to hold ingested recipes (mock DB)
recipe_store = []

### Youtube Scrapping - v2

References: https://colab.research.google.com/drive/1KOlGHd3Rqs_w6_6F_LiqTF1Dcga_XZBk?usp=sharing#scrollTo=jBTs4wnBnH1G

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from urllib.parse import urlparse, parse_qs
import glob

def get_video_id(link: str) -> str:
    parsed = urlparse(link)
    if 'youtube.com' in parsed.netloc:
        return parse_qs(parsed.query).get('v', [''])[0]
    elif 'youtu.be' in parsed.netloc:
        return parsed.path.lstrip('/')
    return ""

def try_youtube_captions(video_id: str) -> str:
    try:
        transcript = YouTubeTranscriptApi().get_transcript(video_id)
        return " ".join([t["text"] for t in transcript])
    except TranscriptsDisabled:
        print("⚠️ Captions are disabled for this video.")
    except NoTranscriptFound:
        print("⚠️ No transcript found.")
    except Exception as e:
        print(f"❌ Caption extraction failed: {e}")
    return ""

def transcribe_audio_with_yt_dlp(link: str, use_cookies: bool = False, cookies_path: str = "youtube_cookies.txt") -> str:
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': False,
        'outtmpl': '/tmp/audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    if use_cookies:
        ydl_opts['cookiefile'] = cookies_path

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])

        audio_files = glob.glob("/tmp/audio*.mp3")
        if not audio_files:
            raise FileNotFoundError("No audio file found.")
        audio_path = audio_files[0]

        print(f"🎧 Transcribing audio: {audio_path}")
        result = whisper_model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        print(f"❌ Audio transcription failed: {e}")
        return ""

def ingest_recipe_smart(link: str, cookies_path: str = "youtube_cookies.txt") -> str:
    video_id = get_video_id(link)
    if not video_id:
        return "Invalid YouTube link."

    print("🔍 Trying captions...")
    text = try_youtube_captions(video_id)
    if text:
        print("✅ Captions found.")
        return text

    print("🌀 Trying audio transcription without cookies...")
    text = transcribe_audio_with_yt_dlp(link, use_cookies=False)
    if text:
        print("✅ Transcription without cookies succeeded.")
        return text

    print("🔐 Trying with cookies...")
    text = transcribe_audio_with_yt_dlp(link, use_cookies=True, cookies_path=cookies_path)
    if text:
        print("✅ Transcription with cookies succeeded.")
        return text

    return "❌ All attempts failed."

In [ ]:
# TEST-v2
# link = "https://youtu.be/2uYoqclu6so"
link = "https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ"
text = ingest_recipe_smart(link)
print(text[:500])

#### YouTube Scrapping - v1

In [ ]:
import glob
import os
import shutil

def is_youtube_link(link: str) -> bool:
    parsed = urlparse(link)
    return "youtube.com" in parsed.netloc or "youtu.be" in parsed.netloc

def transcribe_youtube(link: str, cookies_path: str = "youtube_cookies.txt") -> str:
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': False,
        'outtmpl': '/tmp/audio.%(ext)s',  # Allow yt-dlp to set proper extension
        'cookiefile': cookies_path,
        'postprocessors': [{  # Extract audio as mp3
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])

        # Find the downloaded mp3 file in /tmp
        files = glob.glob("/tmp/audio*.mp3")
        if not files:
            raise FileNotFoundError("Audio file not found after download")
        audio_path = files[0]

        print(f"Using audio file: {audio_path}")

        result = whisper_model.transcribe(audio_path)
        return result['text']

    except Exception as e:
        print(f"❌ Error downloading/transcribing: {str(e)}")
        return ""



def extract_recipe_from_text(text: str) -> Dict:
    steps = re.split(r"\d+\.", text)
    steps = [step.strip() for step in steps if len(step.strip()) > 10]
    return {
        "raw_text": text,
        "steps": steps,
        "num_steps": len(steps),
        "tags": [],  # Placeholder for later
    }


def store_recipe(recipe_data: Dict):
    embedding = embedding_model.encode(recipe_data['raw_text'])
    recipe_data['embedding'] = embedding
    recipe_store.append(recipe_data)


In [ ]:
def ingest_recipe_from_youtube(link: str, cookies_path: str = "youtube_cookies.txt"):
    if not is_youtube_link(link):
        return "Invalid YouTube URL."

    text = transcribe_youtube(link, cookies_path)
    if not text:
        return "Failed to transcribe video."

    display(Markdown(f"### Transcribed Text\n{text[:500]}..."))

    recipe = extract_recipe_from_text(text)
    store_recipe(recipe) # Use store_recipe here

    display(Markdown("### Extracted Steps"))
    for i, step in enumerate(recipe['steps']):
        display(Markdown(f"**Step {i+1}:** {step}"))

    return recipe

In [ ]:
# Sample test link
youtube_link = "https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ"
result = ingest_recipe_from_youtube(youtube_link)

### Instagram Scrapping

In [ ]:
!pip install instaloader

In [ ]:
import instaloader

# Create instaloader instance
L = instaloader.Instaloader(download_video_thumbnails=False, download_geotags=False, download_comments=False)


In [ ]:
# Only needed if you want to access private content or saved reels
# L.login('your_username', 'your_password')  # Optional

In [ ]:
def extract_instagram_text(link: str) -> str:
    shortcode = None
    match = re.search(r"instagram\.com/(reel|p)/([a-zA-Z0-9_-]+)", link)
    if match:
        shortcode = match.group(2)
    else:
        return "Invalid Instagram link."

    post = instaloader.Post.from_shortcode(L.context, shortcode)

    caption = post.caption or "No caption available."
    video_url = post.video_url if post.is_video else None

    # Optional: download video audio for Whisper transcription
    if video_url:
        audio_path = "/tmp/insta_audio.mp3"
        os.system(f"yt-dlp -x --audio-format mp3 -o '{audio_path}' '{video_url}'")
        transcript = whisper_model.transcribe(audio_path)['text']
        return f"{caption}\n\nTranscribed Audio:\n{transcript}"
    else:
        return caption

In [ ]:
def ingest_recipe_from_instagram(link: str):
    print("Fetching Instagram content...")
    text = extract_instagram_text(link)
    display(Markdown(f"### Extracted/Transcribed Text\n{text[:500]}..."))

    recipe = extract_recipe_from_text(text)
    store_recipe(recipe)

    display(Markdown("### Extracted Steps"))
    for i, step in enumerate(recipe['steps']):
        display(Markdown(f"**Step {i+1}:** {step}"))

    return recipe

In [ ]:
# Example Reel/Post
insta_link = "https://www.instagram.com/reel/DGtu1YbPwd9/?igsh=MTBhcDk1Ymk2ZzFoYg=="
result = ingest_recipe_from_instagram(insta_link)

### Text Scrapping

In [ ]:
def ingest_recipe_from_text(raw_text: str):
    display(Markdown("### Input Recipe Text"))
    display(Markdown(raw_text[:500] + ("..." if len(raw_text) > 500 else "")))

    recipe = extract_recipe_from_text(raw_text)
    recipe['raw_text'] = raw_text  # Ensure raw_text is included
    recipe['embedding'] = embedding_model.encode(raw_text)

    # store_recipe_to_vector_db(recipe)

    display(Markdown("### Extracted Steps"))
    for i, step in enumerate(recipe['steps']):
        display(Markdown(f"**Step {i+1}:** {step}"))

    return recipe

In [ ]:
# Paste or type any recipe here
sample_text = """
Masala French Toast

1. Break 2 eggs into a bowl.
2. Add 1 chopped green chili, 1/2 tsp red chili powder, salt, and some chopped cilantro.
3. Mix well with a fork.
4. Dip bread slices into the mixture.
5. Toast both sides on a buttered pan until golden brown.
6. Serve hot with ketchup or chutney.
"""

recipe = ingest_recipe_from_text(sample_text)


### Store Recipes in FAISS + Metadata Index

This step needs to be before ingestion

After ingesting any recipe from any source, it should store it in FAISS immediately for future reference.

STEPS:

1. Setup the FAISS
2. Store the Ingested Recipies from different sources in FAISS

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import numpy as np

# Define embedding dimension (must match your SentenceTransformer)
embedding_dim = 384  # for 'all-MiniLM-L6-v2'

# FAISS index (Flat L2 search for simplicity)
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Metadata store for structured search (can replace with SQLite later)
metadata_store = []


In [ ]:
def store_recipe_to_vector_db(recipe: Dict):
    embedding = recipe['embedding']
    if isinstance(embedding, list):
        embedding = np.array(embedding).astype('float32')

    # Add to FAISS
    faiss_index.add(np.expand_dims(embedding, axis=0))

    # Add to metadata store
    metadata_store.append({
        "text": recipe['raw_text'],
        "steps": recipe['steps'],
        "num_steps": recipe['num_steps'],
        "tags": recipe.get("tags", []),
        # Add more filters later like cuisine, type, time etc.
    })

    print(f"✅ Recipe stored! Total stored: {len(metadata_store)}")


In [ ]:
store_recipe_to_vector_db(recipe)

In [ ]:
# After ingestion:
# YT_recipe = ingest_recipe_from_youtube("https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ")
# store_recipe_to_vector_db(YT_recipe)
store_recipe_to_vector_db(result)

In [ ]:
# Test function - This is done by the LLM after it gets the query

def search_similar_recipes(query: str, top_k: int = 3):
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')

    distances, indices = faiss_index.search(query_embedding, top_k)

    print("🔍 Top Similar Recipes:\n")
    for i, idx in enumerate(indices[0]):
        meta = metadata_store[idx]
        print(f"🔸 Match {i+1}: {meta['text'][:300]}...\n---\n")


In [ ]:
# Test function - This is done by the LLM after it gets the query

# search_similar_recipes("I want to make salmon with lime under 30 minutes", top_k=1)
search_similar_recipes("I want to make eggs under 30 minutes", top_k=1)

### Use RAG using Gemini

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
def generate_rag_response_with_gemini(user_query: str, top_k: int = 3) -> str:
    # Embed query and search FAISS
    query_embedding = embedding_model.encode([user_query])
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = faiss_index.search(query_embedding, top_k)

    # Build context
    context_snippets = []
    for idx in indices[0]:
        recipe = metadata_store[idx]
        snippet = f"Steps:\n" + "\n".join(recipe['steps'][:6])
        context_snippets.append(snippet)

    context = "\n\n---\n\n".join(context_snippets)

    prompt = f"""You are a helpful recipe assistant.

User request: {user_query}

Here are some recipes:

{context}

Please recommend the best-fitting recipe and explain why it was chosen.
"""

    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text


In [ ]:
query = "Looking for a light Italian appetizer under 30 minutes"
response = generate_rag_response_with_gemini(query)
print(response)